In [1]:
import numpy as np
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [2]:
# Define image dimensions and batch size
img_height, img_width = 128, 128
batch_size = 32
# Define directories for training and testing data
train_data_dir = "../dataset/train"
test_data_dir = "../dataset/test"

In [3]:
# Data augmentation for training images
train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1./255  # Normalize pixel values
)

# Data augmentation for testing images (only rescale)
test_datagen = ImageDataGenerator(rescale=1./255)


In [4]:
# Create data generators for training and testing
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # No need to shuffle for evaluation
)



Found 1174 images belonging to 14 classes.
Found 428 images belonging to 14 classes.


# TRAINING PHASE

In [5]:
# Define the VGG16 base model
def create_EfficientNetV2L_model():
    EfficientNetV2L_model = EfficientNetV2L(
        weights='imagenet', 
        include_top=False, 
        input_shape=(img_height, img_width, 3)
    )
    EfficientNetV2L_model.trainable = False


    model = Sequential([
        EfficientNetV2L_model,
        GlobalAveragePooling2D(),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(14, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    print(model.summary())
    return model




In [6]:
#MODEL fit
model = create_EfficientNetV2L_model()
model.fit(train_generator, epochs=2, validation_data=test_generator)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-l (Functiona  (None, 4, 4, 1280)       117746848 
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 64)                81984     
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                        

In [7]:
# predicting train dataset 
train_pridiction = model.predict(train_generator)

# true labels 
train_true_labels = train_generator.classes




37/37 [==============================] - 31s 376ms/step


In [8]:
print( train_pridiction.shape)
print( train_true_labels.shape)

#adding the true labels to the pridiction
train_pridiction = np.concatenate((train_pridiction, train_true_labels.reshape(-1,1)), axis=1)
print( train_pridiction.shape)


(1174, 14)
(1174,)
(1174, 15)


In [9]:
#saving predictions in csv file
np.savetxt("../model_saved_files/predictions/EfficientNetV2L_predictions.csv", train_pridiction, delimiter=",")

# Save the model
# model.save("../model_saved_files/models_weights/EfficientNetV2L.h5")

### applying fuzzy using random forest

In [18]:
# applying random forest on the predictions
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

df=train_pridiction
X = df[:,:-1]
Y = df[:,-1]


clf = RandomForestClassifier()
clf.fit(X,Y)

RandomForestClassifier()

# TESTING

In [19]:
# applying neural network 
test_prediction = model.predict(test_generator)
test_true_labels = test_generator.classes


14/14 [==============================] - 4s 281ms/step


In [23]:
#applying random forest on the predictions
result=clf.predict(test_prediction)


In [24]:
#printing the classification report
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
print(classification_report(test_true_labels, result))
print(accuracy_score(test_true_labels, result))


              precision    recall  f1-score   support

           0       0.04      0.06      0.05        17
           1       0.04      0.17      0.07        18
           2       0.09      0.08      0.09        24
           3       0.17      0.06      0.09       103
           4       0.03      0.05      0.04        19
           5       0.27      0.22      0.24        18
           6       0.00      0.00      0.00        15
           7       0.10      0.09      0.09        23
           8       0.04      0.06      0.05        16
           9       0.04      0.11      0.05        18
          10       0.19      0.09      0.12       104
          11       0.00      0.00      0.00        17
          12       0.05      0.06      0.05        18
          13       0.00      0.00      0.00        18

    accuracy                           0.07       428
   macro avg       0.07      0.07      0.07       428
weighted avg       0.12      0.07      0.08       428

0.07476635514018691
